In [55]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup as BS
import numpy as np
import json
from statistics import mode
from scipy import stats
import json
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [3]:
connection = sqlite3.connect("database.sqlite")

cur = connection.cursor()

In [107]:
cur.execute("""SELECT *
                FROM Matches
                WHERE Date BETWEEN '2010-12-31' and '2011-12-31'
                JOIN Unique_Teams
                ON Matches.HomeTeam = Unique_Teams.TeamName;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
hometeams = df['HomeTeam'].unique()
df

OperationalError: near "JOIN": syntax error

In [108]:
cur.execute("""SELECT *
                FROM Matches
                WHERE Date BETWEEN '2010-12-31' and '2011-12-31'
                ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
hometeams = df['HomeTeam'].unique()
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,481,D1,2010,2011-01-15,Wolfsburg,Bayern Munich,1,1,D
1,483,D1,2010,2011-04-23,Ein Frankfurt,Bayern Munich,1,1,D
2,485,D1,2010,2011-03-05,Hannover,Bayern Munich,3,1,H
3,489,D1,2010,2011-01-29,Werder Bremen,Bayern Munich,1,3,A
4,490,D1,2010,2011-02-05,FC Koln,Bayern Munich,3,2,H
...,...,...,...,...,...,...,...,...,...
1003,44677,E0,2011,2011-12-31,Chelsea,Aston Villa,1,3,A
1004,44678,E0,2011,2011-12-31,Man United,Blackburn,2,3,A
1005,44679,E0,2011,2011-12-31,Norwich,Fulham,1,1,D
1006,44680,E0,2011,2011-12-31,Stoke,Wigan,2,2,D


In [148]:
cur.execute("""SELECT *
                FROM Matches
                ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
s11 = df[df['Season']==2011]
germs = s11[s11['Div']!='E0']
for date in germs['Date']:
    time = date
    url = ''
    weather = requests.get(url)
    

        

In [161]:
query = """
        SELECT *
        FROM Matches
        """
# JOIN Teams_in_Matches ON Matches.Match_ID = Teams_in_Matches
df = pd.DataFrame(cur.execute(query).fetchall())
df.columns = [x[0] for x in cur.description]
s11 = df[df['Season']==2011]
s11germ = s11[s11['Div']!='E0']
lat = 52.5065133
long = 13.1445703
weather = []
# for date in s11germ['Date']:
# #     print(date)
#     time = date
#     url='https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.format(apikey,lat,long,time)
# #     print(url)
#     response = requests.get(url)
#     weather.append(response.text)
# weather

In [47]:
# WINNER
cur.execute("""SELECT *
                FROM Teams_in_Matches
                JOIN Unique_Teams
                USING (Unique_Team_ID)
                JOIN Matches
                USING (Match_ID)
                WHERE Season == '2011' AND Div != 'E0'
                ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,Match_ID,Unique_Team_ID,TeamName,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,1,Bayern Munich,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1092,22,Nurnberg,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
2,1093,1,Bayern Munich,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
3,1093,17,Stuttgart,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
4,1094,1,Bayern Munich,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
...,...,...,...,...,...,...,...,...,...,...,...
1219,1701,25,Fortuna Dusseldorf,D2,2011,2011-08-15,Ein Frankfurt,Fortuna Dusseldorf,1,1,D
1220,1702,25,Fortuna Dusseldorf,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D
1221,1702,34,Aachen,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D
1222,1703,25,Fortuna Dusseldorf,D2,2011,2011-09-16,Erzgebirge Aue,Fortuna Dusseldorf,2,4,A


In [59]:
date = df['Date']
teams = df['TeamName'].unique()
teams = list(teams)
lat = 52.5065133
long = 13.1445703
weather = []
for i in date.iloc[::2]:
    time = i
    keys = get_keys('/Users/nielsclausen/.secret/DARKSKYAPI.json')
    api_key = keys['api_key']
    url = 'https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.format(api_key,lat,long,time)
#     response = requests.get(url)
#     weather.append(response.text)

    

In [71]:

teams = df['TeamName'].unique()
goals = []
for x, i in teams:
    if x==df.loc[i, 'TeamName']:
        if x==df.loc[i, 'HomeTeam']:
            goals += df.loc[i, 'FTHG']
        elif x==df.loc[i, 'AwayTeam']:
            goals += df.loc[i, 'FTAG']
            


ValueError: too many values to unpack (expected 2)

object = teams
run it through so that each team has stats x weather

In [163]:
for date in s11germ['Date']:
    time = date
    url = 'https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.format(apikey,lat,long,time)
    print(url)

NameError: name 'apikey' is not defined

In [42]:
date = df['Date']
for i in date:
    time = i

